In [2]:
import pytesseract
import cv2
import sys
import numpy as np

In [32]:
img = cv2.imread("../datasets/Manga109/Manga109_released_2021_12_30/images/AisazuNihaIrarenai/005.jpg")
kernel = np.ones((2, 2))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# thresh, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
# img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
# cv2.imshow("name", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
tess_config = "-l jpn_vert --psm 3"

h, w = img.shape
d = pytesseract.image_to_data(img, config=tess_config, output_type=pytesseract.Output.DICT)
n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("output/ocr-test.jpg", img)

True